In [ ]:
import sys
sys.path.insert(1, 'C:/Users/nconroy/Documents/EventClustering/POC/')


In [ ]:
import csv
import corpus
import pandas as pd

In [ ]:
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 20)

In [ ]:
testfile = "data/new_articles.csv"

In [ ]:
with open(testfile, 'r') as read_obj:
        # pass the file object to DictReader() to get the DictReader object
        dict_reader = csv.DictReader(read_obj)
        # get a list of dictionaries from dct_reader
        flat_test = list(dict_reader)

In [ ]:
lang = 'eng'
basedataset = corpus.Corpus()
basedataset.load_corpora(r"../news-clustering/dataset/dataset.dev.json",
                          r"../news-clustering/dataset/clustering.dev.json", set([lang]))
#basedataset.load_corpora(r"../news-clustering/dataset/dataset.test.json",
#                           r"../news-clustering/dataset/clustering.test.json", set([lang]))

In [ ]:
#input_data = flat_test
input_data = basedataset.documents 


In [ ]:
print (input_data[3])

In [ ]:
#stest_cluster_pool= pd.read_pickle("models/aggregator_testTHR_dev.pkl")
test_cluster_pool= pd.read_pickle("models/aggregator_NNent_dev.pkl")

In [ ]:
#test_scores= utils.pd.read_pickle("models/scores_testTHR_dev.pkl")
test_scores= pd.read_pickle("models/scores_NNent_dev.pkl")

In [ ]:
#test_times = utils.pd.read_pickle("models/timesNN.pkl")
test_times = pd.read_pickle("models/times_NNent_dev.pkl")

In [ ]:
#print (len(cluster_pool))
print (len(test_cluster_pool.clusterpool))

In [ ]:
def get_accuracy(cluster_pool):
    for cluster in cluster_pool:
        for id in cluster.ids:
            print (next(item['cluster'] for item in input_data if item["id"] == id))

In [ ]:
ids = []
doc_count = 0
for clst in test_cluster_pool.clusterpool:
    print (clst.ids)
    doc_count = doc_count +len(clst.ids)
    ids.append(clst.ids)

In [ ]:
print(doc_count)

<b>Inspect individual documents in those clusters</b>

In [ ]:
for id in ['23309595', '23236595', '23302375', '23250759', '23254816']:
    print ('\n\n', next((item['title'], item['date'], 'actual cluster: ', item['cluster']) for item in input_data if item["id"] == id))
    print ('\n The matched cscore is ', test_scores[id][1][0])

In [ ]:
import matplotlib.pyplot as plt



cluster_size = list(test_times.keys())           
clustering_time = list(test_times.values())       

plt.plot(cluster_size,clustering_time)
plt.title('Clustering time by cluster pool size')
plt.xlabel('Clusterpool size')
plt.ylabel('Clustering time')
plt.show()

In [ ]:
print(lang,"#docs",len(test_corpus.documents))


In [ ]:
df_testcorpus = pd.DataFrame(test_corpus.documents)
df_testcorpus = df_testcorpus[:2500]


In [ ]:
print (test_corpus)

Shuffle the base test dataset

In [ ]:
df_basecorpus = pd.DataFrame(base_corpus.documents)
df_basecorpus.head()

In [ ]:
df_testcorpus.describe()


Convert new document data to representations for matching (if there is none saved)

In [ ]:
#Create appropriate text/token representations for the document set
rep_df = get_base(df_testcorpus)
rep_df = add_tokenized(rep_df)
rep_df = add_lemmas(rep_df)
rep_df = add_entities(rep_df)
    
#Generate the tf/idf vectors from the appropriate model
docrep_dicts = get_tfidf(rep_df)

Keep them for posterity

In [ ]:
#SAVE THE TEST CLUSTER REPRESENTATIONS WITH PICKLE
with open('saved_models/test_document_reps.pickle','wb') as f:
    pickle.dump(docrep_dicts, f)


Load the saved test corpus and shuffle it

In [ ]:
docrep_dicts= pd.read_pickle("saved_models/test_document_reps.pickle")

In [ ]:
import random
random.shuffle(docrep_dicts)

Get maximum c-cscore for candidates in the cluster pool

In [ ]:
def get_cscore_max(doc, cluster_pool):
    cscores = {}
    for ind, clust in enumerate(cluster_pool):
        cscores[ind]=c_score(clust,doc)
    mx_val = max(cscores.values())
    mx_ind = max(cscores, key=cscores.get)

    targ_clust = cluster_pool[mx_ind]
    return (targ_clust, mx_val )


In [ ]:
import time

#for doc in randomly shuffled documents:
for rawdoc in df_testcorpus.sample(frac=1).itertuples():
    t0 = time.time()
    #Create representations for entire base training set.
    rep_ = get_base(pd.DataFrame([rawdoc]))
    rep_df = add_tokenized(rep_)
    rep_df = add_lemmas(rep_df)
    rep_df = add_entities(rep_df)
    doc = get_tfidf(rep_df)[0]
    
    #Find the maximum cscore in cluster pool candidates and get the actual text from the rep
    clust, score = get_cscore_max(doc, cluster_pool)
    t1 = time.time()
    
    doc_row = df_testcorpus.loc[df_testcorpus['id'] == doc['id']]
    
    #Look at two random elements from the matching cluster
    clustdoc_ids = random.sample(clust['idset'],2)
    
    print ("\n\nANCHOR DOC TITLE: ", doc_row['title'].values)
    print ("CLOSEST CLUSTER: ", clust['cluster'])
    print ("C-SCORE: ", score)
    print ("CLUSTER SAMPLES: ")
    for id in clustdoc_ids:
        base_row = df_basecorpus.loc[df_basecorpus['id'] == id]
        print (id, base_row['title'].values)
    print ("CONVERT AND MATCH TIME: ", t1-t0)
